In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class dataset(Dataset):
    def __init__(self,length = 64):
        super(dataset).__init__()
        self.ls = np.linspace(0,18,19)
        self.length = length
    def __getitem__(self, index):
        # np.random.seed(index)
        mult = self.ls[index%19]/3
        x = np.linspace(0,int(3.14*400-1),int(3.14*400))
        a = np.random.random()*5
        b = np.random.random()*17000
        c = np.random.random()*4
        y = a * np.sin(mult*(x+b)) + c
        return torch.tensor(y).reshape(-1,1).to(torch.float),self.ls[index%19]
    def __len__(self):
        return self.length
ds = dataset()
dl = DataLoader(ds,batch_size=16,shuffle=True)

In [3]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) 
        
        # Decode the hidden state of the last time step
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out
    

In [4]:

# example usage
input_dim = 1 # assuming single channel
hidden_dim = 128
num_layers = 2
dropout = 0.2
num_classes = 19

model = LSTMClassifier(input_dim, hidden_dim, num_layers, dropout, num_classes)

In [5]:
# Import necessary libraries
import torch.optim as optim

# Define the model, criterion and optimizer
model = LSTMClassifier(input_dim, hidden_dim, num_layers, dropout, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define some training parameters
num_epochs = 500

# Define a function to calculate the accuracy of the model
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

# Begin training
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for i,(ts,cl) in enumerate(dl):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(ts)
        loss = criterion(outputs, cl.to(torch.long))
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update running loss and accuracy
        running_loss += loss.item()
        running_acc += accuracy(outputs, cl)
        
    # Print statistics for the current epoch
    print('Epoch: {} Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, running_loss/len(dl), running_acc/len(dl)))

print('Training complete')


Epoch: 1 Loss: 2.9463 Acc: 0.0469
Epoch: 2 Loss: 2.9431 Acc: 0.0469
